# DRL usage example

First we import al the used libraries, remember to always import `Sinergym` even if it says is not used, because that is needed to define the environments

In [1]:
import sinergym
from sinergym.utils.callbacks import LoggerEvalCallback
from sinergym.utils.rewards import *
from sinergym.utils.wrappers import LoggerWrapper
from datetime import datetime
import gym
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import CallbackList
from stable_baselines3.common.vec_env import DummyVecEnv

First let's define some strings and variables for the execution.

In [ ]:
environment = "Eplus-demo-v1"
episodes = 4
experiment_date = datetime.today().strftime('%Y-%m-%d %H:%M')

# register run name
name = F"DQN-{environment}-episodes_{episodes}({experiment_date})"

Now we are ready to create the Gym Environment. Here we use the run name defined before as well as the type of reward, in owr case we are going to use the LinearReward defined by `Sinergym`. You can define your own or use any of the other defined by `Sinergym` have a look at ref:`rewards` for more information on that.

In [ ]:
env = gym.make(environment, reward=LinearReward)

We can add also a Wrapper to the environment, we are going to use a Logger (extension of ``gym.Wrapper``) this is used to monitor and log the interactions with the environment and save the data into a CSV.

In [ ]:
env = LoggerWrapper(env)

At this point we have the environment all set up and ready to be used to define and create our learning model in this case it's going to be a DQN, but we can use any other (have a look at the `DRL_battery.py` and read :ref:`Deep Reinforcement Learning Integration` for more detailed information on available DRL algorithms).
Please feel free to play and change the values of the attributes of our model (or even the model) to see the differences.

In [ ]:
model = DQN('MlpPolicy', env, verbose=1)

Now we need to calculate the number of timesteps of each episode for the evaluation.

In [ ]:
n_timesteps_episode = env.simulator._eplus_one_epi_len / \
                      env.simulator._eplus_run_stepsize


Now we need to create a vectorized wrapper for the environment because the callbacks we are going to use requiere a vector.

In [ ]:
env_vec = DummyVecEnv([lambda: env])

We are going to use the LoggerEval callback to print and save the training process.

In [ ]:
callbacks = []

# Set up Evaluation and saving best model
eval_callback = LoggerEvalCallback(
    env_vec,
    best_model_save_path='best_model/' + name + '/',
    log_path='best_model/' + name + '/',
    eval_freq=n_timesteps_episode * 2,
    deterministic=True,
    render=False,
    n_eval_episodes=2)
callbacks.append(eval_callback)

callback = CallbackList(callbacks)

This is the number of total time steps for the training.

In [ ]:
timesteps = episodes * n_timesteps_episode

Now is time to train the model with the callbacks defined earlier. This may take a few minutes, depending on your computer.

In [ ]:
model.learn(
    total_timesteps=timesteps,
    callback=callback,
    log_interval=1)

Now we save the current model.

In [ ]:
model.save(env.simulator._env_working_dir_parent + '/' + name)

And as always, remember to close the environment.

In [ ]:
env.close()